In [5]:
from __future__ import print_function
from __future__ import division

import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from math import sqrt
import sklearn
from sklearn.linear_model import LogisticRegression
import pickle
import time
import torch

from utils import test_accuracy, projection, veccos, binary_search_cx, \
                  find_exp_score, find_slab_score, project_l2_centroid, \
                  project_l2_centroid_straight, project_slab, project_slab_straight,\
                  contaminate_dataset

from attackers import StraightAttack, SemiOnlineAttack, ConcentratedAttack, GreedyAttack

In [8]:
## fully online experiment

def experiment(dataset, taus=None):
    
    filepath = "./data/"+dataset+"/"
    clf = LogisticRegression(fit_intercept=False, solver='liblinear')
    if (taus==None):
        n_taus = 5
    else:
        n_taus = len(taus)
    res = [0 for a in range(n_taus)]
    
    for i in range(n_exp):    
        print ("Running the {}-th experiment".format(i))
        start_time = time.time()
        filename = filepath+str(i)
        with open(filename,"rb") as f:
            datasets = pickle.load(f)

        X_init, Y_init = datasets[0] # generate defense constraints and init w if necessary.
        X_clean, Y_clean = datasets[1] # the clean data stream
        X_valid, Y_valid = datasets[2] # validation set
        X_test, Y_test = datasets[3]   # the actual test set

        clf.fit(X_init, Y_init)
        w_0 = np.zeros((1,d))
        print ("Initial accuracy is {}".format(test_accuracy(X_test, Y_test, w_0)))
        w_t = -clf.coef_
        w_t /= norm(w_t)
        print ("Target accuracy is {}".format(test_accuracy(X_test, Y_test, w_t)))

        if attack_method == "simplistic":
            attacker = StraightAttack()
            attacker.set_param(datasets, w_0, w_t, R, eta, 
                                        defense, n_iter_warmup, n_attack) 
        elif attack_method == "greedy":
            attacker = GreedyAttack()
            attacker.set_param(datasets, w_0, w_t, R, eta, 
                                        defense, n_iter_warmup, n_attack) 
        elif attack_method == "concentrated":
            attacker = ConcentratedAttack()
            attacker.set_param(datasets, w_0, w_t, R, eta, 
                                        defense, n_iter_warmup, n_attack) 
        elif attack_method == "semi-online":
            attacker = SemiOnlineAttack()
            X_adv, Y_adv = X_clean[:n_attack, :], Y_clean[:n_attack]
            attacker.set_param(datasets, w_0, w_t, R, eta, 
                                        defense, n_iter_warmup, n_attack, (-X_adv, Y_adv))

        global loc
        
        attacker.set_init_set(X_init, Y_init)
        if defense_method == "slab":
            scores = attacker.slab_scores()[-1]
        elif defense_method == "norm":
            scores = attacker.l2_norms()[-1]
        elif defense_method == "L2":
            scores = attacker.l2_distances_to_centroid()[-1]
        
        tau_levels = [0.3, 0.5, 0.7, 0.9, 1]
        taus = [scores[int(tau_level*len(scores))-1] for tau_level in tau_levels]
        print (taus)
        w_list = [[] for tau in taus]
        res = [[] for tau in taus]
        accs = [[] for tau in taus]
        online_res = [[] for tau in taus]
        for j,tau in enumerate(taus):

            attacker.set_defense_threshold(defense_method, tau)
            attacker.set_defense_range(defense_range)
            attacker.set_init_set(X_init, Y_init)

            i_clean = 0
            
            for k in range(n_horizon):
                if loc[k]: #attack
                    if attack_method == "semi-online":
                        attacker.set_param_lite(attacker.w_curr, w_t, 0, 1, 
                                           (-X_clean[k].reshape(1,-1), Y_clean[k].reshape(1)))

                    pt = attacker.find_best_poisoning_points()
                    attacker.update_w(pt)
                    online_res[j].append(0)
                else:      #update on clean point
                    pt = (attacker.X_clean[k,:], attacker.Y_clean[k])
                    online_res[j].append(round(test_accuracy(X_clean[k].reshape(1,-1), 
                                                             Y_clean[k].reshape(1),
                                                             attacker.w_curr)))
                    if attacker.meet_constraints(pt):
                        attacker.update_w(pt)
                    i_clean += 1
                acc = test_accuracy(X_test, Y_test, attacker.w_curr)
                w_list[j].append(attacker.w_curr)
                accs[j].append(acc)
                if k%50==0:
                    print ("Accuracy at time {} is {}".format(k, acc))
                
            print ("acc on clean stream is", sum(online_res[j])/i_clean)
                    
            print (tau, sum(accs[j])/n_horizon)
            res[j] = (sum(accs[j])/n_horizon)
            attacker.reset()
            print (res)
            print("--- %s seconds ---" % (time.time() - start_time))
            
        results = [res, online_res, accs, w_list, taus, loc]
        filepath = "_".join(["./results/fully", dataset, attack_method, defense_method, defense_range, str(i)])
        print (filepath)
        with open(filepath, "wb") as f:
            pickle.dump(results, f)
        filepath = "_".join(["./results/fully", dataset, attack_method, defense_method, defense_range, str(i)])
        print (filepath)    
        with open(filepath, "rb") as f:
            results = pickle.load(f)
        res, online_res, accs, w_list, taus, loc = results    
    return w_list, (X_test, Y_test)

In [9]:
R = 10
n_exp = 1
d, eta = 100, 0.01
n_attack, n_clean, n_init, n_test, n_valid = 200, 10000, 5000, 5000, 2000
n_iter_warmup = n_clean
n_horizon=1000
attack_chance = 0.1
loc = (np.random.permutation(n_horizon) < n_horizon*attack_chance)
dataset = "IMDB"


In [10]:
defense_method = "norm"
defense = {defense_method:0}
attack_methods = ["simplistic", "greedy", "semi-online"]
for attack_method in attack_methods:
    defense_range = "all-pts"
    w_list1, (X_test, Y_test) = experiment(dataset)
    res1 = [test_accuracy(X_test, Y_test, w) for w in w_list1[-1]]

Running the 0-th experiment
Initial accuracy is 0.4986
Target accuracy is 0.1156
[3.036725, 3.2409685, 3.4199, 3.6594794, 5.620403]
Accuracy at time 0 is 0.5232
Accuracy at time 50 is 0.1986
Accuracy at time 100 is 0.2254
Accuracy at time 150 is 0.2032
Accuracy at time 200 is 0.2136
Accuracy at time 250 is 0.207
Accuracy at time 300 is 0.2068
Accuracy at time 350 is 0.188
Accuracy at time 400 is 0.2056
Accuracy at time 450 is 0.1724
Accuracy at time 500 is 0.157
Accuracy at time 550 is 0.1704
Accuracy at time 600 is 0.1754
Accuracy at time 650 is 0.1662
Accuracy at time 700 is 0.159
Accuracy at time 750 is 0.1624
Accuracy at time 800 is 0.1646
Accuracy at time 850 is 0.1536
Accuracy at time 900 is 0.1532
Accuracy at time 950 is 0.1518
acc on clean stream is 0.1811111111111111
3.036725 0.18175960000000027
[0.18175960000000027, [], [], [], []]
--- 14.661790609359741 seconds ---
Accuracy at time 0 is 0.5232
Accuracy at time 50 is 0.3146
Accuracy at time 100 is 0.3834
Accuracy at time 150 

Running the 0-th experiment
Initial accuracy is 0.4986
Target accuracy is 0.1156
[3.036725, 3.2409685, 3.4199, 3.6594794, 5.620403]
Accuracy at time 0 is 0.5232
Accuracy at time 50 is 0.1796
Accuracy at time 100 is 0.4004
Accuracy at time 150 is 0.3516
Accuracy at time 200 is 0.3032
Accuracy at time 250 is 0.175
Accuracy at time 300 is 0.2114
Accuracy at time 350 is 0.1164
Accuracy at time 400 is 0.136
Accuracy at time 450 is 0.1178
Accuracy at time 500 is 0.1356
Accuracy at time 550 is 0.2196
Accuracy at time 600 is 0.2304
Accuracy at time 650 is 0.131
Accuracy at time 700 is 0.1484
Accuracy at time 750 is 0.3204
Accuracy at time 800 is 0.1164
Accuracy at time 850 is 0.1602
Accuracy at time 900 is 0.1166
Accuracy at time 950 is 0.1154
acc on clean stream is 0.19555555555555557
3.036725 0.2050337999999997
[0.2050337999999997, [], [], [], []]
--- 30.971158742904663 seconds ---
Accuracy at time 0 is 0.5232
Accuracy at time 50 is 0.3498
Accuracy at time 100 is 0.4686
Accuracy at time 150 

In [11]:
defense_method = "L2"
defense = {defense_method:0}

for attack_method in attack_methods:
    defense_range = "all-pts"
    w_list2, (X_test, Y_test) = experiment(dataset)
    res2 = [test_accuracy(X_test, Y_test, w) for w in w_list2[-1]]

Running the 0-th experiment
Initial accuracy is 0.4986
Target accuracy is 0.1156
[3.0101087, 3.2124045, 3.3975067, 3.6377218, 5.5455184]
Accuracy at time 0 is 0.5232
Accuracy at time 50 is 0.221
Accuracy at time 100 is 0.3016
Accuracy at time 150 is 0.2456
Accuracy at time 200 is 0.2734
Accuracy at time 250 is 0.2552
Accuracy at time 300 is 0.264
Accuracy at time 350 is 0.2476
Accuracy at time 400 is 0.2624
Accuracy at time 450 is 0.2202
Accuracy at time 500 is 0.1828
Accuracy at time 550 is 0.206
Accuracy at time 600 is 0.211
Accuracy at time 650 is 0.2146
Accuracy at time 700 is 0.193
Accuracy at time 750 is 0.2068
Accuracy at time 800 is 0.1988
Accuracy at time 850 is 0.183
Accuracy at time 900 is 0.1796
Accuracy at time 950 is 0.172
acc on clean stream is 0.21333333333333335
3.0101087 0.22210960000000013
[0.22210960000000013, [], [], [], []]
--- 13.806117057800293 seconds ---
Accuracy at time 0 is 0.5232
Accuracy at time 50 is 0.4094
Accuracy at time 100 is 0.4898
Accuracy at time 

Running the 0-th experiment
Initial accuracy is 0.4986
Target accuracy is 0.1156
[3.0101087, 3.2124045, 3.3975067, 3.6377218, 5.5455184]
Accuracy at time 0 is 0.5232
Accuracy at time 50 is 0.2138
Accuracy at time 100 is 0.5518
Accuracy at time 150 is 0.3942
Accuracy at time 200 is 0.414
Accuracy at time 250 is 0.2594
Accuracy at time 300 is 0.2608
Accuracy at time 350 is 0.1248
Accuracy at time 400 is 0.1828
Accuracy at time 450 is 0.1224
Accuracy at time 500 is 0.1788
Accuracy at time 550 is 0.3624
Accuracy at time 600 is 0.3494
Accuracy at time 650 is 0.1772
Accuracy at time 700 is 0.1674
Accuracy at time 750 is 0.4322
Accuracy at time 800 is 0.1262
Accuracy at time 850 is 0.1708
Accuracy at time 900 is 0.1174
Accuracy at time 950 is 0.1162
acc on clean stream is 0.27555555555555555
3.0101087 0.2653233999999995
[0.2653233999999995, [], [], [], []]
--- 34.86647152900696 seconds ---
Accuracy at time 0 is 0.5232
Accuracy at time 50 is 0.462
Accuracy at time 100 is 0.5674
Accuracy at tim

In [12]:
defense_method = "slab"
defense = {defense_method:0}

for attack_method in attack_methods:
    defense_range = "all-pts"
    w_list3, (X_test, Y_test) = experiment(dataset)
    res3 = [test_accuracy(X_test, Y_test, w) for w in w_list3[-1]]

Running the 0-th experiment
Initial accuracy is 0.4986
Target accuracy is 0.1156
[0.119148225, 0.20944604, 0.3233255, 0.51220363, 1.3213937]
Accuracy at time 0 is 0.4986
Accuracy at time 50 is 0.6648
Accuracy at time 100 is 0.735
Accuracy at time 150 is 0.7482
Accuracy at time 200 is 0.7602
Accuracy at time 250 is 0.7666
Accuracy at time 300 is 0.7646
Accuracy at time 350 is 0.7698
Accuracy at time 400 is 0.789
Accuracy at time 450 is 0.7898
Accuracy at time 500 is 0.8024
Accuracy at time 550 is 0.81
Accuracy at time 600 is 0.8076
Accuracy at time 650 is 0.8018
Accuracy at time 700 is 0.8044
Accuracy at time 750 is 0.809
Accuracy at time 800 is 0.8108
Accuracy at time 850 is 0.8154
Accuracy at time 900 is 0.814
Accuracy at time 950 is 0.8142
acc on clean stream is 0.7755555555555556
0.119148225 0.7786419999999987
[0.7786419999999987, [], [], [], []]
--- 15.470307111740112 seconds ---
Accuracy at time 0 is 0.5232
Accuracy at time 50 is 0.732
Accuracy at time 100 is 0.7856
Accuracy at ti

[0.119148225, 0.20944604, 0.3233255, 0.51220363, 1.3213937]
Accuracy at time 0 is 0.4986
Accuracy at time 50 is 0.5882
Accuracy at time 100 is 0.6344
Accuracy at time 150 is 0.6238
Accuracy at time 200 is 0.6372
Accuracy at time 250 is 0.6472
Accuracy at time 300 is 0.6514
Accuracy at time 350 is 0.6524
Accuracy at time 400 is 0.6624
Accuracy at time 450 is 0.6646
Accuracy at time 500 is 0.6686
Accuracy at time 550 is 0.681
Accuracy at time 600 is 0.685
Accuracy at time 650 is 0.6832
Accuracy at time 700 is 0.6864
Accuracy at time 750 is 0.6894
Accuracy at time 800 is 0.6916
Accuracy at time 850 is 0.694
Accuracy at time 900 is 0.693
Accuracy at time 950 is 0.6954
acc on clean stream is 0.6511111111111111
0.119148225 0.6630403999999996
[0.6630403999999996, [], [], [], []]
--- 66.62542533874512 seconds ---
Accuracy at time 0 is 0.5232
Accuracy at time 50 is 0.625
Accuracy at time 100 is 0.6748
Accuracy at time 150 is 0.6664
Accuracy at time 200 is 0.6864
Accuracy at time 250 is 0.6924
A